<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/hsv-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-05-30 17:26:20--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8100 (7.9K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   7.91K  --.-KB/s    in 0s      

2022-05-30 17:26:20 (105 MB/s) - ‘utils.py’ saved [8100/8100]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [5]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub


from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [6]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x3/1'

In [7]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [8]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [9]:
classes, class_weight = get_class_weight()

In [10]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [11]:
hsv_creator = ModelCreator(HUB_URL, 'hsv-model')

In [12]:
hsv_model = hsv_creator.make_model()
callbacks = hsv_creator.get_callbacks()

In [ ]:
history = hsv_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=5,
                        class_weight=class_weight)

Epoch 1/5
519/519 [==============================] - 7050s 13s/step - loss: 367.6065 - precision: 0.5090 - recall: 0.5049 - auc: 0.3812 - accuracy: 0.5061
Epoch 2/5
519/519 [==============================] - 2400s 5s/step - loss: 252.4337 - precision: 0.5879 - recall: 0.5862 - auc: 0.4622 - accuracy: 0.5869
Epoch 3/5
519/519 [==============================] - 2417s 5s/step - loss: 200.1027 - precision: 0.6237 - recall: 0.6218 - auc: 0.5058 - accuracy: 0.6227
Epoch 4/5
304/519 [================>.............] - ETA: 16:34 - loss: 173.9472 - precision: 0.6424 - recall: 0.6398 - auc: 0.5348 - accuracy: 0.6404

In [ ]:
history_2 = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=10,
                          class_weight=class_weight,
                          initial_epoch=5)